In [ ]:
# use fsnippets to find permit location based on lat and long
    # column names: latitude, longitude
    # filter when none given
# calculate activity in neighborhood by adding together square footage contaminated
    # column name: square feet
    #filter when none given

In [5]:
#imports
import pandas as pd
import fpsnippets
import geopandas

#importing relevant columns
asb = pd.read_csv("asbestos_dataset.csv", usecols = ['city', 'square_feet', 'latitude','longitude'])

#filtering out no given lat and long
asb = asb[~asb.latitude.isnull()]

#filtering out no given square feet
asb = asb[~asb.square_feet.isnull()]
asb[1:10]
#make a 2D array of neighborhoods and square footage of asbestos?

#filtering out not in a pittsburgh neighborhood
#for row in asb:
    #determine neighborhood of latitude and longitude
    #rownum = 0
    #lat = asb.iloc[rownum, 2]
    #long = asb.iloc[rownum, 3]
    #neighborhood = fpsnippets.geo_to_neighborhood(lat,long)
    
    #parse neighborhoods and associated square footage of asbestos with them
    #if neighborhood != 'None'
        #if in 2D array
            #add sq footage from this row to 2D array
        #else
            #add neighborhood to 2d array 
            #add square footage of row to 2d array
    #rownum = rownum +1


,city,square_feet,latitude,longitude
1,MCKEESPORT,1000.0,40.345639,-79.878971
2,MCKEESPORT,1000.0,40.352548,-79.873915
3,PITTSBURGH,1200.0,40.443036,-80.024855
4,NaN,2100.0,40.584681,-79.785346
8,PITTSBURGH,8000.0,40.441819,-80.000835
9,PITTSBURGH,3000.0,40.330668,-79.944172
10,PITTSBURGH,4725.0,40.440230,-79.998988
11,PITTSBURGH,73330.0,40.440804,-79.998628
12,PITTSBURGH,250.0,40.454664,-79.939587
